In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import cohen_kappa_score, mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import spearmanr
from scipy.optimize import minimize
import os
import joblib

In [2]:
### Define targets
ordinal_targets = ['ADNC', 'Braak', 'Thal', 'CERAD']
continuous_targets = ['percent 6e10 positive area', 'percent AT8 positive area', 'percent GFAP positive area', 'percent NeuN positive area']
all_target_columns = ordinal_targets + continuous_targets

In [3]:
### Test model
def test_model(df, target, region):
    
    ## Load trained model
    model = joblib.load(f'./models/{region}/{region}_{target}_model.pkl')
    
    ## Prepare data (drop other targets and create feature matrix)
    other_targets = [t for t in all_target_columns if t != target]
    X = df.drop(columns=['Donor ID', target] + other_targets)
    y = df[target].values
    donor_ids = df['Donor ID'].values
    
    ## Make predictions (continuous values)
    y_pred = model.predict(X)
    
    ## Calculate metrics based on target type (according to DREAM tasks)
    
    ## For ordinal targets
    if target in ordinal_targets:
        
        ## Load optimized thresholds from trained model
        optimized_thresholds = model.optimized_thresholds_
        
        ## Apply thresholds to get ordinal predictions
        y_classes = np.digitize(y_pred, bins=optimized_thresholds)
        
        ## Calculate metrics for ordinal targets
        metrics = {
            'quadratic_weighted_kappa': cohen_kappa_score(y, y_classes, weights='quadratic'),
            'mean_absolute_error': mean_absolute_error(y, y_classes),
            'spearman_correlation': spearmanr(y, y_classes)[0],
            'thresholds_used': str(optimized_thresholds.round(3))
        }
        
        ## Calculate predictions for ordinal targets
        predictions_df = pd.DataFrame({
            'Donor ID': donor_ids,
            'true_value': y,
            'predicted_value': y_classes
        })

    else:
        
        ## Calculate metrics for continuous targets
        mean_t, mean_p = np.mean(y), np.mean(y_pred)
        var_t, var_p = np.var(y, ddof=1), np.var(y_pred, ddof=1)
        ccc = (2 * np.cov(y, y_pred)[0,1]) / (var_t + var_p + (mean_t - mean_p)**2)
        
        metrics = {
            'concordance_correlation_coefficient': ccc,
            'mean_squared_error': mean_squared_error(y, y_pred),
            'r2_score': r2_score(y, y_pred)
        }
        
        ## Calculate predictions for continuous targets
        predictions_df = pd.DataFrame({
            'Donor ID': donor_ids,
            'true_value': y,
            'predicted_value': y_pred
        })
    
    ## Save predictions
    predictions_df.to_csv(f'./output/{region}/{region}_{target}_test_predictions.csv', index=False)
    
    ## Save metrics
    metrics_df = pd.DataFrame(list(metrics.items()), columns=['metric_name', 'metric_value'])
    metrics_df.to_csv(f'./output/{region}/{region}_{target}_test_metrics.csv', index=False)

### Process all test datasets
for csv_file in os.listdir('data/'):
    if csv_file in ['dataset_mtg_ordered.csv', 'dataset_a9_ordered.csv']:
        
        region = 'MTG' if 'mtg' in csv_file else 'A9'
        df = pd.read_csv(f'data/{csv_file}')
        
        for target in all_target_columns:
            test_model(df, target, region)

In [4]:
### Process all test datasets
for csv_file in os.listdir('data/'):
    if csv_file in ['dataset_mtg_ordered.csv', 'dataset_a9_ordered.csv']:
        
        region = 'MTG' if 'mtg' in csv_file else 'A9'
        df = pd.read_csv(f'data/{csv_file}')
        
        for target in all_target_columns:
            test_model(df, target, region)